## Assignment 2: Adversarial Training

This assignment requires you to create adversarial examples. You will do adversarial training, i.e., train the model with sets of adversarial examples you generated and evaluate the performances of the model on test sets.

### What is Adversarial Training?
Adversarial training is a machine learning technique to improve models' robustness by training them on adversarial examples. Adversarial examples are input data that has been intentionally modified to cause the model to misclassify or produce an incorrect output. 

When a model is trained using adversarial examples, it becomes more resilient to adversarial attacks and is able to better identify and classify input data that may have been modified or corrupted. This may lead to improved performance of the model in real-world scenarios where the input data may not always be perfect.

However, adversarial training can also have some negative impacts on ML models. For example, it can lead to overfitting, where the model becomes too specialized to the particular adversarial examples used in training and is unable to generalize well to new examples. Additionally, adversarial training can increase the computational requirements of training the model due to the need for generating adversarial examples.

Overall, while adversarial training can improve the robustness of ML models, it is important to carefully consider its potential benefits and drawbacks and to evaluate the trade-offs in terms of model performance and computational requirements. The following are the steps involved in adversarial training:

1. Generate adversarial examples: In the first step, we generate adversarial examples by perturbing the original data points in such a way that the modifications are small and not noticeable to humans but are enough to cause misclassification by the neural network.

2. Train on adversarial examples: In the second step, we train the neural network on adversarial examples in addition to the original training data. This helps to improve the network's ability to recognize and classify adversarial examples correctly.

3. Evaluate performance: In the third step, we evaluate the performance of the network on both the original and adversarial test data. This helps to determine if the adversarial training has improved the network's robustness against adversarial attacks.

Overall, adversarial training is a powerful technique that can help improve the security and reliability of deep neural networks.

In this Homework, you will run different adversarial training algorithms on the ResNet18 model with the adversarial examples, and evaluating the model performances on test data. The goal is to get experience in generating adversarial examples and train the model with these examples, i.e., adversarial training.

We have provided the model architecture ($\texttt{model.py}$) and some pre-defined functions ($\texttt{utils.py}$) so you can import and use them directly in the notebook.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
from model import ResNet18
from utils import trades_loss, mixup_data, mixup_criterion, make_dataloader, eval_test

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


### Q1 (20 points)
Use the following parameters to define the LinfPGDAttack():

- Epsilon: 8/255
- PGD Steps: 10
- PGD Step Size: 0.003

In [2]:
class LinfPGDAttack(nn.Module):
    def __init__(self, model, epsilon, steps=10, step_size=0.003):
        super().__init__()
        self.model = model
        self.epsilon = epsilon
        self.steps = steps
        self.step_size = step_size

    def perturb(self, x_natural, y):
        """
        Computes the gradient of the cross-entropy loss with respect to the input
        image `x_adv` and updates the image based on the gradient direction. The 
        perturbation is clipped to ensure it stays within a specified epsilon range
        and is finally clamped to ensure pixel values are valid. 
        
        The resulting perturbed image is returned.
        """
        # *********** Your code starts here ***********
        x_adv = x_natural.clone().detach().requires_grad_(True)
        for _ in range(self.steps):
            self.model.zero_grad()
            with torch.enable_grad():  
                pred_y = self.model(x_adv)
                loss = F.cross_entropy(pred_y, y)
            loss.backward()
            x_adv = x_adv + self.step_size * x_adv.grad.sign()
            x_adv = torch.min(torch.max(x_adv, x_natural - self.epsilon), x_natural + self.epsilon)
            x_adv = torch.clamp(x_adv, 0, 1)
        
            # Prepare x_adv for the next iteration
            x_adv = x_adv.detach().requires_grad_(True)

        # *********** Your code ends here *************
        return x_adv.detach()


    def forward(self, x_natural, y):
        x_adv = self.perturb(x_natural, y)
        return x_adv

There are many implementations of adversarial training; in this assignment, we ask you to evalaute which training algorithm can make the model more robust to LinfPGDAttack().

In [3]:
def train_ep(model, train_loader, mode, pgd_attack, optimizer, criterion, epoch, batch_size):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        if mode == 'natural':
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        elif mode == 'adv_train': # [Ref] https://arxiv.org/abs/1706.06083
            model.eval()
            adv_x = pgd_attack(inputs, targets)
            model.train()

            optimizer.zero_grad()
            outputs = model(adv_x)
            loss = criterion(outputs, targets)

        elif mode == 'adv_train_trades': # [Ref] https://arxiv.org/abs/1901.08573
            optimizer.zero_grad()
            loss = trades_loss(model=model, x_natural=inputs, y=targets, optimizer=optimizer)
            
        elif mode == 'adv_train_mixup': # [Ref] https://arxiv.org/abs/1710.09412
            model.eval()
            benign_inputs, benign_targets_a, benign_targets_b, benign_lam = mixup_data(inputs, targets)            
            adv_x = pgd_attack(inputs, targets)
            adv_inputs, adv_targets_a, adv_targets_b, adv_lam = mixup_data(adv_x, targets)
            
            model.train()
            optimizer.zero_grad()
            
            benign_outputs = model(benign_inputs)
            adv_outputs = model(adv_inputs)
            loss_1 = mixup_criterion(criterion, benign_outputs, benign_targets_a, benign_targets_b, benign_lam)
            loss_2 = mixup_criterion(criterion, adv_outputs, adv_targets_a, adv_targets_b, adv_lam)
            
            loss = (loss_1 + loss_2) / 2

        else:
            print("No training mode specified.")
            raise ValueError()

        loss.backward()
        optimizer.step()

        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{:05d}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(inputs), len(train_loader) * batch_size,
                       100. * (batch_idx + 1) / len(train_loader), loss.item()))

In [5]:
def train(model, train_loader, val_loader, pgd_attack,
          mode='natural', epochs=25, batch_size=256, learning_rate=0.1, momentum=0.9, weight_decay=2e-4,
          checkpoint_path='model1.pt'):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

    best_acc = 0
    for epoch in range(epochs):
        # training
        train_ep(model, train_loader, mode, pgd_attack, optimizer, criterion, epoch, batch_size)

        # evaluate clean accuracy
        test_loss, test_acc = eval_test(model, val_loader)

        # remember best acc@1 and save checkpoint
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)

        # save checkpoint if is a new best
        if is_best:
            torch.save(model.state_dict(), checkpoint_path)
        print('================================================================')

### Q2 (40 points)
Use the four training modes ("natural", "adv_train", "adv_train_trades", and "adv_train_mixup") to obtain four models, and save them as $\texttt{model1.pt}$, $\texttt{model2.pt}$, $\texttt{model3.pt}$, and $\texttt{model4.pt}$, respectively.

When calculating your losses, you may encounter Nan. In this case, you may consider adjusting the `learning rate` to solve the problem.

In [6]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [7]:
# define parameters
batch_size = 256
data_path = './dataset/' # directory of the data
epsilon = 8/255
steps = 10
epochs = 1

# create data loader
train_loader, val_loader = make_dataloader(data_path, batch_size)
model = ResNet18()
model.to(device)
pgd_attack = LinfPGDAttack(model, epsilon, steps)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
training_mode = "natural"

# Define Model and Launch Training (Define Adversary If You Need It)
model = ResNet18().to(device)

# Write your code here
train(model, train_loader, val_loader, pgd_attack,
          mode=training_mode, epochs=epochs, batch_size=batch_size, learning_rate=0.1, momentum=0.9, weight_decay=2e-4,
          checkpoint_path='model1.pt')

Train Epoch: 0 [00256/50176 (1%)]	 Loss: 2.361035


KeyboardInterrupt: 

In [10]:
training_mode = "adv_train"

# Define Model and Launch Training (Define Adversary If You Need It)
model = ResNet18().to(device)

# Write your code here
train(model, train_loader, val_loader, pgd_attack,
          mode=training_mode, epochs=epochs, batch_size=batch_size, learning_rate=0.1, momentum=0.9, weight_decay=2e-4,
          checkpoint_path='model2.pt')


Train Epoch: 0 [00256/50176 (1%)]	 Loss: 2.468390
Train Epoch: 0 [13056/50176 (26%)]	 Loss: 2.156428
Train Epoch: 0 [25856/50176 (52%)]	 Loss: 1.972983
Train Epoch: 0 [38656/50176 (77%)]	 Loss: 1.898687
Test: Average loss: 0.0072, Accuracy: 3442/10000 (34%)
Train Epoch: 1 [00256/50176 (1%)]	 Loss: 1.775255
Train Epoch: 1 [13056/50176 (26%)]	 Loss: 1.701212
Train Epoch: 1 [25856/50176 (52%)]	 Loss: 1.638372
Train Epoch: 1 [38656/50176 (77%)]	 Loss: 1.621612
Test: Average loss: 0.0063, Accuracy: 4438/10000 (44%)
Train Epoch: 2 [00256/50176 (1%)]	 Loss: 1.496722
Train Epoch: 2 [13056/50176 (26%)]	 Loss: 1.401162
Train Epoch: 2 [25856/50176 (52%)]	 Loss: 1.386252
Train Epoch: 2 [38656/50176 (77%)]	 Loss: 1.216043
Test: Average loss: 0.0076, Accuracy: 3937/10000 (39%)
Train Epoch: 3 [00256/50176 (1%)]	 Loss: 1.241223
Train Epoch: 3 [13056/50176 (26%)]	 Loss: 1.252075
Train Epoch: 3 [25856/50176 (52%)]	 Loss: 0.969843
Train Epoch: 3 [38656/50176 (77%)]	 Loss: 1.045187
Test: Average loss: 0.0

In [13]:
training_mode = "adv_train_trades"

# Define Model and Launch Training (Define Adversary If You Need It)
model = ResNet18().to(device)

# Write your code here
train(model, train_loader, val_loader, pgd_attack,
          mode=training_mode, epochs=epochs, batch_size=batch_size, learning_rate=0.05, momentum=0.9, weight_decay=2e-4,
          checkpoint_path='model3.pt')

Train Epoch: 0 [00256/50176 (1%)]	 Loss: 2.453964
Train Epoch: 0 [13056/50176 (26%)]	 Loss: 1.896858
Train Epoch: 0 [25856/50176 (52%)]	 Loss: 1.826412
Train Epoch: 0 [38656/50176 (77%)]	 Loss: 1.668022
Test: Average loss: 0.0089, Accuracy: 3135/10000 (31%)
Train Epoch: 1 [00256/50176 (1%)]	 Loss: 1.484552
Train Epoch: 1 [13056/50176 (26%)]	 Loss: 1.525583
Train Epoch: 1 [25856/50176 (52%)]	 Loss: 1.541090
Train Epoch: 1 [38656/50176 (77%)]	 Loss: 1.391539
Test: Average loss: 0.0050, Accuracy: 5566/10000 (56%)
Train Epoch: 2 [00256/50176 (1%)]	 Loss: 1.392904
Train Epoch: 2 [13056/50176 (26%)]	 Loss: 1.228774
Train Epoch: 2 [25856/50176 (52%)]	 Loss: 1.252674
Train Epoch: 2 [38656/50176 (77%)]	 Loss: 1.270619
Test: Average loss: 0.0043, Accuracy: 6313/10000 (63%)
Train Epoch: 3 [00256/50176 (1%)]	 Loss: 1.260016
Train Epoch: 3 [13056/50176 (26%)]	 Loss: 1.212505
Train Epoch: 3 [25856/50176 (52%)]	 Loss: 1.273879
Train Epoch: 3 [38656/50176 (77%)]	 Loss: 1.327008
Test: Average loss: 0.0

In [ ]:
training_mode = "adv_train_mixup"

# Define Model and Launch Training (Define Adversary If You Need It)
model = ResNet18().to(device)

# Write your code here
train(model, train_loader, val_loader, pgd_attack,
          mode=training_mode, epochs=epochs, batch_size=batch_size, learning_rate=0.1, momentum=0.9, weight_decay=2e-4,
          checkpoint_path='model4.pt')

### Q3 (20 points)
Use eval_robust() to evaluate each model's robustness against LinfPGDAttack().

In [9]:
from utils import eval_robust

# *********** Your code starts here ***********pgd_attack
# model = ResNet18().to(device)
# model.load_state_dict(torch.load('model1.pt'))
robust_loss, robust_acc = eval_robust(model, val_loader, )
print(f'Training Mode: Natural, robust_loss: {robust_loss}, robust_acc: {robust_acc}')
# *********** Your code ends here *************

In [ ]:
# *********** Your code starts here ***********
model = ResNet18().to(device)
model.load_state_dict(torch.load('model2.pt'))
robust_loss, robust_acc = eval_robust(model, val_loader, pgd_attack)
print(f'Training Mode: adv_train, robust_loss: {robust_loss}, robust_acc: {robust_acc}')
# *********** Your code ends here *************

In [ ]:
# *********** Your code starts here ***********
model = ResNet18().to(device)
model.load_state_dict(torch.load('model3.pt'))
robust_loss, robust_acc = eval_robust(model, val_loader, pgd_attack)
print(f'Training Mode: adv_train_trades, robust_loss: {robust_loss}, robust_acc: {robust_acc}')
# *********** Your code ends here *************

In [ ]:
# *********** Your code starts here ***********
model = ResNet18().to(device)
model.load_state_dict(torch.load('model4.pt'))
robust_loss, robust_acc = eval_robust(model, val_loader, pgd_attack)
print(f'Training Mode: adv_train_mixup, robust_loss: {robust_loss}, robust_acc: {robust_acc}')
# *********** Your code ends here *************

### Q4 (10 points)
Visualize 10 adversarial examples from (with ground truth labels and model's predictions) from the model having best robust accuarcy.

In [ ]:
# *********** Your code starts here ***********



# *********** Your code ends here *************

### Q5 (10 points)
Which adversarial training algorithm achieves the best robust accuracy in only 25 training epochs? Why do you think that adversarial training algorithm outperforms others? Do you encounter any difficulties when you implement this assignment?

*Write your answer here.*

## Submission Instructions

Please submit this  to Canvas, as well as results, as per instructions. 
Please compress `First_Middle_Last_HW1/` into one zip file with the name `First_Middle_Last_HW1.zip` before uploading it to Canvas. The directory contains your notebook and four model checkpoints. As listed below:

- $\texttt{Assignment_2.ipynb}$: your code
- $\texttt{model1.pt}$: your model checkpoint with *natural training* 
- $\texttt{model2.pt}$: your model checkpoint with *adv_train* training
- $\texttt{model3.pt}$: your model checkpoint with *adv_train_trades* training
- $\texttt{model4.pt}$: your model checkpoint with *adv_train_mixup* training

## Academic Integrity

This homework assignment must be done individually. Sharing code or model specifications is strictly prohibited. Homework discussions are allowed only on Piazza, according to the policy outlined on the course web page: [https://canvas.dartmouth.edu/courses/63219](https://canvas.dartmouth.edu/courses/63219). You are not allowed to search online for auxiliary software, reference models, architecture specifications, or additional data to solve the homework assignment. Your submission must be entirely your own work. That is, the code and the answers that you submit must be created, typed, and documented by you alone, based exclusively on the materials discussed in class, and released with the homework assignment. You can obviously consult the class slides posted in Canvas, your lecture notes, and the textbook. Important: the models you will submit for this homework assignment must be
trained exclusively on the specified data provided with this assignment. You can, of course, play with other datasets in your spare time. These rules will be strictly enforced, and any violation will be treated seriously.